In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# import necessary libraries
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score, roc_auc_score

from imblearn.over_sampling import SMOTE
from collections import Counter

# **Data & Pre-processing**

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Masters Thesis/data/attribution_data.csv')
df.head()

,cookie,time,interaction,conversion,conversion_value,channel
0,00000FkCnDfDDf0iC97iC703B,2018-07-03T13:02:11Z,impression,0,0.0,Instagram
1,00000FkCnDfDDf0iC97iC703B,2018-07-17T19:15:07Z,impression,0,0.0,Online Display
2,00000FkCnDfDDf0iC97iC703B,2018-07-24T15:51:46Z,impression,0,0.0,Online Display
3,00000FkCnDfDDf0iC97iC703B,2018-07-29T07:44:51Z,impression,0,0.0,Online Display
4,0000nACkD9nFkBBDECD3ki00E,2018-07-03T09:44:57Z,impression,0,0.0,Paid Search


In [ ]:
df['time'] = pd.to_datetime(df['time'])

In [ ]:
# Extract date
df['date'] = df['time'].dt.date

In [ ]:
# visit_order per cookie based on time order
df = df.sort_values(['cookie', 'time'], ascending=[False, True])
df['visit_order'] = df.groupby('cookie').cumcount() + 1

In [ ]:
df_paths = df.groupby('cookie')['channel'].aggregate(lambda x: x.tolist()).reset_index()
df_last_interaction = df.drop_duplicates('cookie', keep='last')[['cookie', 'conversion']]
df_paths = pd.merge(df_paths, df_last_interaction, how='left', on='cookie')
df_paths.drop(columns = ['cookie'], inplace = True)

In [ ]:
total_conversions = sum(df_paths['conversion'])
print("No. of conversions: ", total_conversions)

No. of conversions:  17639


In [ ]:
# reference: https://www.geeksforgeeks.org/highlight-the-maximum-value-in-each-column-in-pandas/
def highlight_max_attribution(row, df):
    '''
    highlight the maximum in a Series green.
    '''
    is_max = row.loc['Attributed Credit'] == df['Attributed Credit'].max()
    return ['background-color: lightgreen' if is_max else '' for v in row.index]

# **LSTM + Attention (with class weights)**

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Attention
from tensorflow.keras.metrics import AUC

In [ ]:
# Extract sequences and labels
sequences = df_paths['channel'].tolist()
labels = df_paths['conversion'].tolist()

# Tokenization: to convert text into sequences of integers
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequences)
# Converts each sequence in sequences to a sequence of integers based on the tokenizer's vocabulary.
sequences_encoded = tokenizer.texts_to_sequences(sequences)

# Padding
max_len = max(len(seq) for seq in sequences_encoded)
X = pad_sequences(sequences_encoded, maxlen=max_len, padding='post')

# Convert labels to numpy array
y = np.array(labels)

# Vocabulary size: size of the tokenizer's vocabulary + 1 for the padding token.
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Keras Functional API: https://keras.io/guides/functional_api/

In [ ]:
# Model definition
input_layer = Input(shape=(max_len,))
embedding_layer = Embedding(input_dim=vocab_size, output_dim=50, input_length=max_len)(input_layer)
lstm_layer = LSTM(64, return_sequences=True)(embedding_layer)

# Attention mechanism
attention_layer = Attention()([lstm_layer, lstm_layer])

# Flatten the attention output to feed into dense layers
attention_flat = tf.keras.layers.Flatten()(attention_layer)
dense_layer = Dense(32, activation='relu')(attention_flat)
output_layer = Dense(1, activation='sigmoid')(dense_layer)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', AUC(name='auc')])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
# reference: https://stackoverflow.com/questions/46009619/keras-weighted-binary-crossentropy
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

# Convert class weights to a dictionary
class_weight_dict = dict(enumerate(class_weights))

In [ ]:
# Train model
model.fit(X_train, y_train, epochs=5, batch_size=128, class_weight=class_weight_dict, validation_split=0.2)

Epoch 1/5
1201/1201 ━━━━━━━━━━━━━━━━━━━━ 408s 337ms/step - accuracy: 0.5953 - auc: 0.5284 - loss: 0.6892 - val_accuracy: 0.7048 - val_auc: 0.5895 - val_loss: 0.7128
Epoch 2/5
1201/1201 ━━━━━━━━━━━━━━━━━━━━ 391s 326ms/step - accuracy: 0.7466 - auc: 0.5752 - loss: 0.6813 - val_accuracy: 0.6471 - val_auc: 0.5918 - val_loss: 0.7125
Epoch 3/5
1201/1201 ━━━━━━━━━━━━━━━━━━━━ 454s 336ms/step - accuracy: 0.6936 - auc: 0.5799 - loss: 0.6810 - val_accuracy: 0.7545 - val_auc: 0.5921 - val_loss: 0.6636
Epoch 4/5
1201/1201 ━━━━━━━━━━━━━━━━━━━━ 401s 334ms/step - accuracy: 0.7057 - auc: 0.5840 - loss: 0.6806 - val_accuracy: 0.7225 - val_auc: 0.5919 - val_loss: 0.6898
Epoch 5/5
1201/1201 ━━━━━━━━━━━━━━━━━━━━ 441s 333ms/step - accuracy: 0.7230 - auc: 0.5812 - loss: 0.6800 - val_accuracy: 0.7474 - val_auc: 0.5921 - val_loss: 0.6593


In [ ]:
# model.save('/content/drive/My Drive/Masters Thesis/saved_models/lstmAttention_model-balanced.h5')

In [ ]:
model = tf.keras.models.load_model('/content/drive/My Drive/Masters Thesis/saved_models/lstmAttention_model-balanced.h5', custom_objects={'Attention': Attention})

In [ ]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 134)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_2 (Embedding)   │ (None, 134, 50)        │            300 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_2 (LSTM)             │ (None, 134, 64)        │         29,440 │ embedding_2[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_2 (Attention)   │ (None, 134, 64)        │              0 │ lstm_2[0][0],          │
│                           │                        │                │ lstm_2[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_2 (Flatten)       │ (None, 8576)           │              0 │ attention_2[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 32)             │        274,464 │ flatten_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 1)              │             33 │ dense_4[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 912,713 (3.48 MB)

 Trainable params: 304,237 (1.16 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 608,476 (2.32 MB)

In [ ]:
# Evaluate model
results = model.evaluate(X_test, y_test)

1501/1501 ━━━━━━━━━━━━━━━━━━━━ 52s 35ms/step - accuracy: 0.7475 - auc: 0.5867 - loss: 0.6591


In [ ]:
# Print all results
print(f'Loss: {round(results[0],2)}')
print(f'Accuracy: {round(results[1],2)}')
if len(results) > 2:
    print(f'AUC: {round(results[2],2)}')  # If AUC or other metrics are included

Loss: 0.66
Accuracy: 0.75
AUC: 0.59


## Assigning Attribution values

In [ ]:
model.layers[3]

<Attention name=attention_2, built=True>

In [ ]:
attention_model = Model(inputs=model.input, outputs=model.layers[3].output)
attention_weights = attention_model.predict(X_test)

1501/1501 ━━━━━━━━━━━━━━━━━━━━ 43s 28ms/step


In [ ]:
channel_names = [channel.title() for channel in list(tokenizer.word_index.keys())]

In [ ]:
def compute_unnormalized_attributions(X_sequences, y_labels, attention_weights, channel_names):
    # Initialize a dictionary to store total attributions and counts for each channel
    channel_attributions = {channel: 0 for channel in channel_names}
    channel_counts = {channel: 0 for channel in channel_names}

    # Process each sequence
    for i, (sequence, label) in enumerate(zip(X_sequences, y_labels)):
        if label == 1:  # Only consider sequences ending in a conversion
            seq_attention_weights = attention_weights[i]

            # Average attention weights across the sequence length
            avg_attention_weights = np.mean(np.abs(seq_attention_weights), axis=1)
            # print(avg_attention_weights.shape)
            # print(sequence)
            # Map the averaged attention weights to channels
            for j, channel_index in enumerate(sequence):
                if channel_index < len(channel_names):  # Ensure the channel index is valid
                    channel_name = channel_names[channel_index]
                    channel_attributions[channel_name] += avg_attention_weights[j]
                    channel_counts[channel_name] += 1

    # Calculate mean attributions for each channel
    mean_attributions = {channel: channel_attributions[channel] / channel_counts[channel] if channel_counts[channel] > 0 else 0
                         for channel in channel_names}

    return mean_attributions

# Compute unnormalized attributions for each channel
mean_attributions = compute_unnormalized_attributions(X_test, y_test, attention_weights, channel_names)

# Convert to DataFrame for easy visualization
df_mean_attributions = pd.DataFrame(list(mean_attributions.items()), columns=['Channel', 'Mean Attribution'])
df_mean_attributions

,Channel,Mean Attribution
0,Facebook,0.009956
1,Paid Search,0.165434
2,Online Video,0.075584
3,Instagram,0.227018
4,Online Display,0.167731


In [ ]:
# Normalize attribution scores to get relative importance for channels
total_conversions = np.sum(y)
score = df_mean_attributions['Mean Attribution']/ df_mean_attributions['Mean Attribution'].sum()
channel_attribution_credit = score / score.sum() * total_conversions
channel_attribution_percentages = round(score / score.sum() * 100, 2).apply(lambda x: f"{x:.2f}%")

In [ ]:
# Create a DataFrame for presentation of channel attributions
channel_attribution_df = pd.DataFrame({
    'Channel': channel_names,
    'Attribution Score': score,
    'Attributed Credit': channel_attribution_credit,
    'Attribution Percentage': channel_attribution_percentages
})

desired_order = ['Facebook', 'Instagram', 'Online Display', 'Online Video', 'Paid Search']
channel_attribution_df['Channel'] = pd.Categorical(channel_attribution_df['Channel'], categories=desired_order, ordered=True)
channel_attribution_df = channel_attribution_df.sort_values('Channel').reset_index(drop=True)

channel_attribution_df.style.apply(highlight_max_attribution, df=channel_attribution_df, axis=1)

,Channel,Attribution Score,Attributed Credit,Attribution Percentage
0,Facebook,0.015419,271.967907,1.54%
1,Instagram,0.351571,6201.362292,35.16%
2,Online Display,0.259757,4581.859594,25.98%
3,Online Video,0.117054,2064.713351,11.71%
4,Paid Search,0.256199,4519.096856,25.62%


---